## MVP Know Your Transaction (KYT) - Transaction Risk Classification Engine

### Project Overview

This notebook presents a comprehensive implementation of a Transaction Risk Classification Engine for Anti-Money Laundering (AML) compliance in cryptocurrency transactions. The project addresses the critical need for risk assessment of Bitcoin transactions by combining traditional AML indicators with blockchain-specific risk factors.

### Domain Context: Financial AML for Transactions

#### Core Domain Definition
Anti-Money Laundering (AML) for transactions encompasses the comprehensive framework of laws, regulations, procedures, and technological solutions designed to prevent criminals from disguising illegally obtained funds as legitimate income through the global financial system. This domain includes detection, prevention, and reporting of money laundering, terrorist financing, tax evasion, market manipulation, and misuse of public funds.

### Problem Definition: Transaction Risk Classification Engine

#### Problem Statement
Develop a system that assigns risk classifications to cryptocurrency transactions in real-time, integrating traditional AML indicators with blockchain-specific risk factors including wallet clustering, transaction graph analysis, and counterparty reputation scoring.

#### Technical Requirements
- **Problem Type**: Classification 
- **Processing Speed**: Sub-second analysis for high-frequency transactions
- **Difficulty Level**: High - requires complex multi-dimensional data processing
- **Output Format**: Risk binary classification (illicit = 1 /licit = 2)

#### Data Landscape
The system can processes multiple data dimensions:
- Transaction metadata (amounts, timestamps, fees)
- Wallet addresses and clustering information
- Transaction graph relationships and network topology
- Counterparty databases and reputation scores
- Sanctions lists and regulatory databases
- Temporal patterns and behavioral baselines

### References

This notebook implementation is based on the comprehensive research and analysis conducted during the project development phase. The following reference documents were used in the composition of this initial description:

- **Domain Research**: [current-domain.md](domains/current-domain.md) - Contains detailed market analysis, regulatory framework research, and commercial viability assessment for the Financial AML domain;
- **Problem Analysis**: [current-problem.md](problems/current-problem.md) - Provides comprehensive problem refinement, technical requirements analysis, and solution approach evaluation;
- **Dataset Evaluation**: [current-dataset.md](datasets/current-dataset.md) - Documents dataset selection criteria, suitability scoring, and detailed feature analysis for the Elliptic dataset;
- **Dataset Analysis & Preprocessing**: [dataset-analysis-and-preprocessing.ipynb](datasets/scripts/dataset-analysis-and-preprocessing.ipynb) - Comprehensive Jupyter notebook containing Elliptic dataset download, exploratory data analysis, and ML preparation steps;

These reference documents contain the foundational research that informed the technical approach, feature engineering strategy, and implementation decisions reflected in this notebook.

---

This notebook serves as the primary entry point for the MVP KYT implementation and it can run independently, providing both technical implementation and business context for real cryptocurrency transaction risk assessment.

### Import Libraries

Comprehensive installation and import of all required libraries for machine learning procedures.

In [ ]:
%pip install pytorch-tabnet scikeras tensorflow
%pip install xgboost lightgbm catboost
%pip install azure-storage-blob

#Colab setup only
#!git clone https://github.com/zzaia/zzaia-mvp-kyt-pos.git
#%cd zzaia-mvp-kyt-pos

import os
import warnings
import joblib
import json
import sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from optuna.samplers import TPESampler
from copy import deepcopy
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.metrics import average_precision_score
import optuna
from optuna.pruners import MedianPruner
import time

# Add datasets/scripts to Python path
scripts_path = Path("./datasets/scripts")
if str(scripts_path.resolve()) not in sys.path:
    sys.path.insert(0, str(scripts_path.resolve()))

# Add models/scripts to Python path
models_scripts_path = Path("./models/scripts")
if str(models_scripts_path.resolve()) not in sys.path:
    sys.path.insert(0, str(models_scripts_path.resolve()))

# Import Azure utilities
from azure_utils import AzureBlobDownloader
azureClient = AzureBlobDownloader("https://stmvppos.blob.core.windows.net", "mvpkytsup")

# Suppress all warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
tf.get_logger().setLevel('ERROR')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


### Loading pre-processed datasets

Let's load the pre-processed and compressed data from remote and local sources. The dataset is a composition of cryptocurrency transactions in the Bitcoin blockchain. 

It has 166 features, of which 92 features represent local transaction information and another 72 features represent aggregated information from one-hop neighboring transactions (directly linked transactions). Thus, the dataset already has curated information about the relationships between transactions. 

This is very important because money laundering and fraud patterns often involve coordinated transaction clusters and neighborhood patterns.

In [ ]:
# Define paths
dataset_dir = Path("elliptic_bitcoin_dataset")
root_dir = Path("./datasets")
processed_dir = root_dir / "processed" / dataset_dir
root_dir.mkdir(exist_ok=True)

# Download processed data from Azure if not present locally
if not processed_dir.exists() or not any(processed_dir.iterdir()):
    print(f"Local processed directory is empty. Downloading from Azure...")
    azureClient.download_documents("datasets/processed", dataset_dir.name, base_path="./")

# The complete dataset already pre-processed 
df_complete = pd.read_hdf(processed_dir / "df_complete.h5", key="df_complete")
print(f"Loaded from HDF5: {df_complete.shape} - All subsequent operations will use compressed data")

# The filtered labeled dataset already pre-processed
df_labeled = pd.read_hdf(processed_dir / "df_labeled.h5", key="df_labeled")
print(f"Loaded from HDF5: {df_labeled.shape} - All subsequent operations will use compressed data")

# The filtered unlabeled dataset already pre-processed
df_unlabeled = pd.read_hdf(processed_dir / "df_unlabeled.h5", key="df_unlabeled")
print(f"Loaded from HDF5: {df_unlabeled.shape} - All subsequent operations will use compressed data")

# The edges dataset that maps relationships between transaction nodes
df_edges = pd.read_hdf(processed_dir / "df_edges.h5", key="df_edges")
print(f"Loaded from HDF5: {df_edges.shape} - All subsequent operations will use compressed data")

# Summary of all datasets
print(f"\n📊 Dataset Summary:")
print(f"  - Features: {df_complete.shape[0]:,} transactions × {df_complete.shape[1] -2} features")
print(f"  - Labeled: {df_labeled.shape[0]:,} transactions")
print(f"  - Unlabeled: {df_unlabeled.shape[0]:,} transactions")
print(f"  - Edges: {df_edges.shape[0]:,} transaction relationships")

### Machine Learning Strategy

Let's apply machine learning techniques to the labeled dataset portion using supervised learning, and apply the prediction to the unknown unlabeled dataset portion in order to establish a performance baseline for future improvements.

Following these steps:

1. Define overall parameters and make data splits;
2. Define all training models to be used;
3. Define all pipelines to be used during training;
4. Define model parameter distributions for a grid search approach;
5. Define the score function to be used during training;
6. Execute the training;
7. Save all resulting models;
8. Validate all models and select the best models;
9. Use best models to predict unknown data;

**1. Define overall parameters and make data splits**

After splitting the labeled dataset into training and validation (test) sets, let's prepare the training dataset for training and validation using the stratified approach, which generates a fixed number of splits (folds) for the dataset following a fixed proportion of train/test samples. 

The main idea of this approach is to guarantee training without bias toward a specific dataset split because it maintains the same class proportion for each split (fold) generated. We use this technique in labeled and supervised learning, assuming that the dataset's pattern does not have significant changes over time.

In this specific dataset, we consider the timestamp as a grouping factor for transactions but not as a changing factor for the dataset's pattern over time.

In [ ]:
# Defining overall parameters
random_seed = 4354 # PARAMETER: random seed
test_size_split = 0.20 # PARAMETER: test set size
n_stratified_splits = 2 # PARAMETER: number of folds
n_pca_components = 0.95 # PARAMETER: PCA components to keep

np.random.seed(random_seed)

# Prepare data
x_labeled = df_labeled.drop(['class', 'txId'], axis=1)
y_labeled = df_labeled['class']

# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(x_labeled, y_labeled,
    test_size=test_size_split, 
    shuffle=True, 
    random_state=random_seed, 
    stratify=y_labeled) # stratified holdout

# Cross-validation setup to be applied in the training set 
cv = StratifiedKFold(n_splits=n_stratified_splits, 
                     shuffle=True, 
                     random_state=random_seed)

**2. Define all models and pipelines to be used during training**

Let's define which ML pipelines to use during training, by configuring which pre-processing operations and models will be used in the training process. Pipelines also help avoid data leakage by ensuring that feature transformation is applied only to the training dataset portion.          

For feature transformation, the standard scaler was used to ensure the best scale for feature values. By normalizing all values to a common metric, it reduces bias toward feature magnitude and enables subsequent operations to capture important patterns between features without being influenced mainly by their magnitude. All features contribute equally to pattern detection. This is especially important in finance because the difference between feature scales can be significant.

For feature dimensionality reduction, Principal Component Analysis (PCA) was used to reduce from 166 to only 59 features. We aim to reduce feature dimensionality while maximizing the dissimilarity of the original dataset, thus extracting the most discriminative features and improving model training performance. It must be applied after the standard scaler to avoid the mentioned magnitude bias, and should be used in datasets that have a large quantity of features.

In [ ]:
# Import pipeline wrappers
from lr_wrapper import LRWrapper
from knn_wrapper import KNNWrapper
from cart_wrapper import CARTWrapper
from nb_wrapper import NBWrapper
from svm_wrapper import SVMWrapper
from bagging_wrapper import BaggingWrapper
from rf_wrapper import RFWrapper
from et_wrapper import ETWrapper
from ada_wrapper import AdaWrapper
from gb_wrapper import GBWrapper
from voting_soft_wrapper import VotingSoftWrapper
from voting_hard_wrapper import VotingHardWrapper
from stacking_wrapper import StackingWrapper
from bag_knn_wrapper import BagKNNWrapper
from xgboost_wrapper import XGBoostWrapper
from lightgbm_wrapper import LightGBMWrapper
from catboost_wrapper import CatBoostWrapper
from histgb_wrapper import HistGBWrapper
from tabnet_wrapper import TabNetWrapper
from fnn_wrapper import FNNWrapper

# Create pipeline wrapper instances
pipeline_wrappers = []
# Uncomment models as needed for training
pipeline_wrappers.append(LRWrapper(random_seed=random_seed))
pipeline_wrappers.append(NBWrapper(random_seed=random_seed))
pipeline_wrappers.append(KNNWrapper(random_seed=random_seed))
pipeline_wrappers.append(CARTWrapper(random_seed=random_seed))
pipeline_wrappers.append(SVMWrapper(random_seed=random_seed))
pipeline_wrappers.append(BaggingWrapper(random_seed=random_seed))
pipeline_wrappers.append(RFWrapper(random_seed=random_seed))
pipeline_wrappers.append(ETWrapper(random_seed=random_seed))
pipeline_wrappers.append(AdaWrapper(random_seed=random_seed))
pipeline_wrappers.append(GBWrapper(random_seed=random_seed))
pipeline_wrappers.append(VotingSoftWrapper(random_seed=random_seed))
pipeline_wrappers.append(VotingHardWrapper(random_seed=random_seed))
pipeline_wrappers.append(StackingWrapper(random_seed=random_seed))
pipeline_wrappers.append(BagKNNWrapper(random_seed=random_seed))
pipeline_wrappers.append(XGBoostWrapper(random_seed=random_seed))
pipeline_wrappers.append(LightGBMWrapper(random_seed=random_seed))
pipeline_wrappers.append(CatBoostWrapper(random_seed=random_seed))
pipeline_wrappers.append(HistGBWrapper(random_seed=random_seed))
pipeline_wrappers.append(TabNetWrapper(random_seed=random_seed))
pipeline_wrappers.append(FNNWrapper(random_seed=random_seed))

print(f"📦 Loaded {len(pipeline_wrappers)} pipeline wrappers:")
for wrapper in pipeline_wrappers:
    print(f"  - {wrapper.name}")

**3. Define model parameter distributions for a grid search approach**

Let's prepare the parameter distributions for a random grid search, by using a distribution of possible parameter values, so that the training phase can explore the best performance models also from the perspective of their hyperparameters. This is a better solution than the common grid search approach because it can explore a broader hyperparameter space and is often faster. 

Three types of parameter distributions were used:

- uniform: Continuous values with equal probability across a range, used when all values in the range are equally valid;
- loguniform: Continuous values on logarithmic scale (exponential distribution), used when smaller values are often better;
- randint: Discrete integer values with equal probability, used for discrete hyperparameters.

In [ ]:
# Build parameter distributions from wrappers
param_distributions = {}
for wrapper in pipeline_wrappers:
    param_distributions[wrapper.name] = wrapper.get_param_distributions()

print(f"📊 Parameter distributions loaded for {len(param_distributions)} models")
for name in param_distributions:
    print(f"  - {name}: {len(param_distributions[name])} hyperparameters")


**4. Define the score function and the objective function to be used during training**

Let's define the score function that will be used to measure each model's performance. Instead of using just one metric alone, the function enables us to define a weighted multi-metric approach, defining which metrics would be more important for the model's performance. The chosen score is a combination of three important metrics: 

- Recall measures how good the model is at not having false negatives;
- Precision measures how good the model is at not having false positives;  
- Accuracy measures how good the model is at not having false classifications;

In financial transaction risk assessments, it is more important to have fewer false negatives than false positive classifications, because it would be less risky to block a transaction wrongly considered illicit than to not block a transaction wrongly considered licit.

In [ ]:
class AMLScorer:
    """
    Anti-Money Laundering scorer for imbalanced fraud detection.
    
    Combines Matthews Correlation Coefficient (MCC) with cost-sensitive scoring
    to balance detection accuracy and business impact.
    """
    
    def __init__(self, cost_fp=1, cost_fn=10, mcc_weight=0.4, cost_weight=0.6):
        """
        Initialize AML scorer.
        
        Args:
            cost_fp: Cost of false positive (blocking legitimate transaction)
            cost_fn: Cost of false negative (missing illicit transaction)
            mcc_weight: Weight for MCC component (0-1)
            cost_weight: Weight for cost component (0-1)
        """
        self.cost_fp = cost_fp
        self.cost_fn = cost_fn
        self.mcc_weight = mcc_weight
        self.cost_weight = cost_weight
    
    @property
    def metric_equation(self):
        """Get the metric equation as a formatted string."""
        return (
            f"AML Score = {self.mcc_weight} × MCC + {self.cost_weight} × Cost Score\n"
            f"where:\n"
            f"  MCC = Matthews Correlation Coefficient\n"
            f"  Cost Score = 1 - (FP × {self.cost_fp} + FN × {self.cost_fn}) / (N × {self.cost_fn})\n"
            f"  FP = False Positives (licit flagged)\n"
            f"  FN = False Negatives (illicit missed)\n"
            f"  N = Total samples"
        )
    
    def score(self, y_true, y_pred):
        """
        Calculate AML composite score.
        
        Args:
            y_true: True labels (0=licit, 1=illicit)
            y_pred: Predicted labels (0=licit, 1=illicit)
        
        Returns:
            float: Composite score (0-1 range, higher is better)
        """
        # MCC: Handles imbalance naturally
        mcc = matthews_corrcoef(y_true, y_pred)
        
        # Cost-sensitive component
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        total_cost = (fp * self.cost_fp) + (fn * self.cost_fn)
        max_cost = len(y_true) * self.cost_fn
        cost_score = 1 - (total_cost / max_cost)
        
        # Weighted combination
        return self.mcc_weight * mcc + self.cost_weight * cost_score
    
    def create_objective(self, model_name, pipeline, param_dist, X_train, y_train, cv, scorer):
        """
        Create Optuna objective function for hyperparameter optimization.
        
        Args:
            model_name: Name of the model being optimized
            pipeline: Sklearn pipeline to optimize
            param_dist: Dictionary of hyperparameter distributions
            X_train: Training features
            y_train: Training labels
            cv: Cross-validation splitter
            scorer: Sklearn scorer object
        
        Returns:
            Callable objective function for Optuna
        """
        def objective(trial):
            # Get parameter suggestions by calling lambdas with trial
            params = {}
            for param_name, suggest_fn in param_dist[model_name].items():
                params[param_name] = suggest_fn(trial)
            
            # Set pipeline parameters
            pipeline.set_params(**params)
            
            # Perform cross-validation
            scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring=scorer, n_jobs=1)
            
            # Return mean score
            return scores.mean()
        
        return objective


# Instantiate scorer
aml_scorer = AMLScorer(cost_fp=1, cost_fn=10, mcc_weight=0.4, cost_weight=0.6)

# Create sklearn scorer
composite_scorer = make_scorer(aml_scorer.score)

# Print metric equation
print("📊 AML Scoring Metric:")
print("-" * 70)
print(aml_scorer.metric_equation)
print("-" * 70)

**5. Execute the training** [CAN BE SKIPPED > 2h]

Let's execute the training phase using random grid search and execute it in parallel, with cross-validation of all dataset splits (folds) and rank the best models by score function.

The final plot will display all model training samples with their mean and variance performance during training.

In [ ]:
# Training parameters
n_trials = 10 # PARAMETER: number of hyperparameter trials per model
n_jobs = 1 
patience = 0.2*n_trials # PARAMETER: early stopping patience
timeout_seconds = 2 * 60 * 60 # PARAMETER: 2 hours 

class EarlyStoppingCallback:
    """Stop optimization after N trials without improvement or timeout"""
    def __init__(self, patience: int, timeout_seconds: float = None):
        self.patience = patience
        self.timeout_seconds = timeout_seconds
        self.trials_without_improvement = 0
        self.best_value = None
        self.start_time = None
        self.is_timed_out = False
    
    def start_timer(self):
        self.start_time = time.time()
    
    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial) -> None:
        if self.timeout_seconds and self.start_time:
            elapsed_time = time.time() - self.start_time
            if elapsed_time >= self.timeout_seconds:
                self.is_timed_out = True
                study.stop()
                return
        
        if trial.state != optuna.trial.TrialState.COMPLETE:
            return
        
        if self.best_value is None:
            self.best_value = study.best_value
            self.trials_without_improvement = 0
            return
        
        if study.best_value > self.best_value:
            self.best_value = study.best_value
            self.trials_without_improvement = 0
        else:
            self.trials_without_improvement += 1
        
        if self.trials_without_improvement >= self.patience:
            study.stop()

checkpoint_dir = Path("./models/mvp-kyt-sup-main")
checkpoint_dir.mkdir(parents=True, exist_ok=True)

training_models = []
cv_results_all = []
study_results = {}

print(f"🔍 Training {len(pipeline_wrappers)} models (patience={patience}, timeout={timeout_seconds/3600:.1f}h)")
print(f"Checkpoints: {checkpoint_dir}")
print("-" * 60)

for wrapper in pipeline_wrappers:
    name = wrapper.name
    pipe = wrapper.build_pipeline(n_pca_components)
    
    model_path = checkpoint_dir / f"{name}.pkl"
    study_path = checkpoint_dir / f"{name}.study.pkl"
    metadata_path = checkpoint_dir / f"{name}.metadata.json"

    if model_path.exists() and study_path.exists() and metadata_path.exists():
        print(f"Loading checkpoint {name}... ", end="", flush=True)
        try:
            # Load
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
            trained_pipe = joblib.load(model_path)
            study = joblib.load(study_path)
            
            meta_score_mean = metadata['cv_score_mean']
            meta_score_std = metadata['cv_score_std']
            meta_actual_trials = metadata['actual_trials']
            meta_ntrials = metadata['n_trials']
            meta_is_timed_out = metadata['is_timed_out']
            cv_scores = np.array([meta_score_mean, meta_score_std])
            cv_results_all.append(cv_scores)

            print(f"✅ {meta_score_mean:.4f} (±{meta_score_std:.4f}) [{meta_actual_trials}/{meta_ntrials}] is timed out: {meta_is_timed_out}]")
            training_models.append((name, trained_pipe))
            study_results[name] = study
            continue
        except Exception as e:
            print(f"⚠️ Failed: {e}")
            continue
    
    print(f"Training {name}...", end=" ", flush=True)
    
    study = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=random_seed),
        pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)
    )
    
    objective = aml_scorer.create_objective(name, pipe, param_distributions, X_train, y_train, cv, composite_scorer)
    early_stopping = EarlyStoppingCallback(patience=patience, timeout_seconds=timeout_seconds)
    early_stopping.start_timer()
    study.optimize(objective, n_trials=n_trials, callbacks=[early_stopping])
    
    pipe.set_params(**study.best_params)
    cv_scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring=composite_scorer, n_jobs=n_jobs)
    cv_results_all.append(cv_scores)
    pipe.fit(X_train, y_train)
    
    actual_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
    print(f"✅ {study.best_value:.4f} (±{cv_scores.std():.4f}) [{actual_trials}/{n_trials}] is timed out: {early_stopping.is_timed_out}]")

    # Create metadata
    metadata = {
        'model_name': name,
        'model_class': pipe.named_steps[list(pipe.named_steps.keys())[-1]].__class__.__name__,
        'cv_score_mean': float(study.best_value),
        'cv_score_std': float(cv_scores.std()),
        'cv_score_type': aml_scorer.metric_equation,
        'trained_at': datetime.now().isoformat(),
        'actual_trials': actual_trials,
        'n_trials': n_trials,
        'patience': patience,
        'timeout_seconds': timeout_seconds,
        'is_timed_out': early_stopping.is_timed_out,
        'best_params': study.best_params,
        'random_seed': random_seed,
    }
    
    # Save 
    metadata_path = checkpoint_dir / f"{name}.metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    joblib.dump(pipe, model_path, compress=3)
    joblib.dump(study, study_path, compress=3)
    
    training_models.append((name, pipe))
    study_results[name] = study

print("-" * 60)
print(f"✅ {len(training_models)} models ready")

fig = plt.figure(figsize=(25,6))
fig.suptitle('Model Comparison - CV Score Distribution') 
ax = fig.add_subplot(111) 
plt.boxplot(cv_results_all, labels=[name for name, _ in training_models])
ax.set_ylabel('CV AML Score')
ax.tick_params(axis='x', rotation=45)
plt.show()


**6. Load all resulting models**

Retrieve previously trained models from local or remote sources.

In [ ]:
# Model checkpoint management
folder_str = "./models/mvp-kyt-sup-main"
folder_dir = Path(folder_str)

if 'training_models' not in locals() or len(training_models) == 0:
    if not folder_dir.exists() or not any(folder_dir.iterdir()):
        print("📥 Downloading from Azure...")
        if azureClient.download_documents("models", "mvp-kyt-sup-main", base_path="./"):
            training_models = []
            study_results = {}
            for f in sorted(folder_dir.glob('*.pkl')):
                if '.study' not in f.name:
                    name = f.stem
                    pipe = joblib.load(f)
                    study_path = f.with_suffix('.study.pkl')
                    study = joblib.load(study_path) if study_path.exists() else None
                    training_models.append((name, pipe))
                    study_results[name] = study
            print(f"✅ Loaded {len(training_models)} models + studies from Azure")
        else:
            print("❌ No models available")
    else:
        training_models = []
        study_results = {}
        for f in sorted(folder_dir.glob('*.pkl')):
            if '.study' not in f.name:
                name = f.stem
                pipe = joblib.load(f)
                study_path = f.with_suffix('.study.pkl')
                study = joblib.load(study_path) if study_path.exists() else None
                training_models.append((name, pipe))
                study_results[name] = study
        print(f"✅ Loaded {len(training_models)} models + studies from checkpoints")

print(f"💾 {len(training_models)} models in {folder_str}")

**7. Optuna Optimization Analysis**

Visualize Optuna optimization history and parameter importance for each model.

In [ ]:
# Visualize Optuna optimization for top 3 models
top_models = sorted(training_models, key=lambda x: study_results[x[0]].best_value, reverse=True)[:3]

print("📊 Optuna Optimization Analysis for Top 3 Models")
print("-" * 60)

for name, _ in top_models:
    study = study_results[name]
    print(f"\n{name} - Best Score: {study.best_value:.4f}")
    print(f"Best Parameters: {study.best_params}")
    
    # Plot optimization history
    fig = optuna.visualization.plot_optimization_history(study)
    fig.update_layout(title=f"{name} - Optimization History")
    fig.show()
    
    # Plot parameter importances
    try:
        fig = optuna.visualization.plot_param_importances(study)
        fig.update_layout(title=f"{name} - Parameter Importances")
        fig.show()
    except:
        print(f"  (Not enough trials for parameter importance analysis)")
    
print("\n" + "=" * 60)

**8. Validate all models and select the best models**

Let's validate and select the best models by applying all trained pipelines to the previously generated testing set using the multi-metric score function.    

Model validation with an unseen dataset during training can give us an approximate measure of how the model would perform in the real world.

In [ ]:
# Select best models based on composite score and PR-AUC
test_results = []
for name, pipe in training_models:
    y_pred = pipe.predict(X_test)
    
    # Primary metric (same as training)
    primary_score = aml_scorer.score(y_test, y_pred)
    
    # Supplementary metric: PR-AUC (threshold-independent)
    try:
        y_proba = pipe.predict_proba(X_test)[:, 1]  # Probability of illicit class
        prauc = average_precision_score(y_test, y_proba)
    except AttributeError:
        # Some models don't support predict_proba
        prauc = 0.0
    
    test_results.append((name, primary_score, prauc))

# Sort by primary metric (composite score)
test_results.sort(key=lambda x: x[1], reverse=True)

print(f"\n🏆 Final Model Rankings:")
print('-'*60)
print(f"{'Model':<15} {'Composite Score':<18} {'PR-AUC':<10}")
print('-'*60)
for name, comp_score, pr_auc in test_results:
    print(f"{name:<15} {comp_score:>8.4f}          {pr_auc:>8.4f}")

**9. Threshold Tuning Analysis**

Optimize the classification threshold to minimize licit false positives while maintaining high illicit detection rate (recall). This analysis helps determine the production-ready threshold for deployment.

In [ ]:
# Cell: Threshold Tuning Analysis

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, confusion_matrix, roc_curve

# Get best model from validation results
best_model_name = test_results[0][0]
best_model = next(model for name, model in training_models if name == best_model_name)

# Generate probability predictions
try:
    y_proba = best_model.predict_proba(X_test)[:, 1]  # Illicit class probabilities
    
    print(f"🎚️ Threshold Tuning Analysis for {best_model_name}")
    print("="*70)
    
    # Evaluate multiple thresholds
    thresholds_to_test = np.linspace(0.1, 0.9, 80)
    threshold_results = []
    
    for thresh in thresholds_to_test:
        y_pred_tuned = (y_proba >= thresh).astype(int)
        
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_tuned).ravel()
        
        # Calculate metrics
        precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # False positive rate
        f1 = 2 * (precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) > 0 else 0
        
        threshold_results.append({
            'threshold': thresh,
            'precision': precision_val,
            'recall': recall_val,
            'f1': f1,
            'fpr': fpr,
            'false_positives': fp,
            'false_negatives': fn,
            'true_positives': tp,
            'true_negatives': tn
        })
    
    df_thresholds = pd.DataFrame(threshold_results)
    
    # Find optimal threshold: Minimize licit false positives while maintaining 85% recall
    target_recall = 0.85
    candidates = df_thresholds[df_thresholds['recall'] >= target_recall]
    
    if len(candidates) > 0:
        optimal_row = candidates.loc[candidates['precision'].idxmax()]
        optimal_threshold = optimal_row['threshold']
        
        print(f"\n📊 Optimal Threshold: {optimal_threshold:.3f}")
        print(f"   Target: Maintain ≥{target_recall*100:.0f}% recall, maximize precision")
        print(f"\n   Performance Metrics:")
        print(f"   ├─ Precision: {optimal_row['precision']:.3f} (minimize licit FPs)")
        print(f"   ├─ Recall: {optimal_row['recall']:.3f} (catch illicit transactions)")
        print(f"   ├─ F1 Score: {optimal_row['f1']:.3f}")
        print(f"   └─ FPR: {optimal_row['fpr']:.3f} (licit misclassification rate)")
        print(f"\n   Confusion Matrix:")
        print(f"   ├─ True Positives (illicit caught): {int(optimal_row['true_positives'])}")
        print(f"   ├─ True Negatives (licit correct): {int(optimal_row['true_negatives'])}")
        print(f"   ├─ False Positives (licit flagged): {int(optimal_row['false_positives'])} ⚠️")
        print(f"   └─ False Negatives (illicit missed): {int(optimal_row['false_negatives'])} ❌")
        
        # Compare with default threshold (0.5)
        default_row = df_thresholds.iloc[(df_thresholds['threshold'] - 0.5).abs().argsort()[:1]].iloc[0]
        
        print(f"\n   Improvement over default threshold (0.5):")
        print(f"   ├─ Precision gain: {(optimal_row['precision'] - default_row['precision'])*100:+.1f}%")
        print(f"   ├─ FP reduction: {int(default_row['false_positives'] - optimal_row['false_positives'])} fewer licit transactions flagged")
        print(f"   └─ Recall change: {(optimal_row['recall'] - default_row['recall'])*100:+.1f}%")
    else:
        optimal_threshold = 0.5
        print(f"⚠️  Could not achieve {target_recall*100:.0f}% recall target. Using default threshold 0.5")
    
    # Visualization: Precision-Recall-Threshold curves
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Precision-Recall curve
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_proba)
    axes[0, 0].plot(recall_curve, precision_curve, linewidth=2, color='blue')
    axes[0, 0].axhline(y=optimal_row['precision'], color='red', linestyle='--', label=f'Optimal (thresh={optimal_threshold:.2f})')
    axes[0, 0].axvline(x=optimal_row['recall'], color='red', linestyle='--')
    axes[0, 0].set_xlabel('Recall (Illicit Detection Rate)', fontsize=12)
    axes[0, 0].set_ylabel('Precision (Accuracy of Illicit Predictions)', fontsize=12)
    axes[0, 0].set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Threshold vs Precision/Recall
    axes[0, 1].plot(df_thresholds['threshold'], df_thresholds['precision'], label='Precision', linewidth=2, color='green')
    axes[0, 1].plot(df_thresholds['threshold'], df_thresholds['recall'], label='Recall', linewidth=2, color='orange')
    axes[0, 1].axvline(x=optimal_threshold, color='red', linestyle='--', linewidth=2, label=f'Optimal={optimal_threshold:.2f}')
    axes[0, 1].set_xlabel('Classification Threshold', fontsize=12)
    axes[0, 1].set_ylabel('Score', fontsize=12)
    axes[0, 1].set_title('Precision & Recall vs Threshold', fontsize=14, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: False Positives vs Threshold
    axes[1, 0].plot(df_thresholds['threshold'], df_thresholds['false_positives'], linewidth=2, color='red')
    axes[1, 0].axvline(x=optimal_threshold, color='blue', linestyle='--', linewidth=2, label=f'Optimal={optimal_threshold:.2f}')
    axes[1, 0].set_xlabel('Classification Threshold', fontsize=12)
    axes[1, 0].set_ylabel('False Positives (Licit Flagged)', fontsize=12)
    axes[1, 0].set_title('Licit False Positives vs Threshold', fontsize=14, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: ROC Curve
    fpr_roc, tpr_roc, _ = roc_curve(y_test, y_proba)
    axes[1, 1].plot(fpr_roc, tpr_roc, linewidth=2, color='purple')
    axes[1, 1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    axes[1, 1].set_xlabel('False Positive Rate', fontsize=12)
    axes[1, 1].set_ylabel('True Positive Rate (Recall)', fontsize=12)
    axes[1, 1].set_title('ROC Curve', fontsize=14, fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 Deployment Recommendation:")
    print(f"   Use threshold = {optimal_threshold:.3f} for prediction:")
  

except AttributeError:
    print(f"⚠️  Model {best_model_name} does not support probability predictions.")
    print(f"   Threshold tuning requires predict_proba() method.")
    print(f"   Consider using models like: XGB, LGB, CAT, SVM(probability=True), etc.")

**10. Use best models to predict unknown data**

Let's apply the best pipeline model to unknown data—data that does not have labels—and display the results to get an idea of how the landscape of unknown illicit transactions could be. This measure can also be used for comparison with future improvements to the machine learning techniques.

In [ ]:
# Apply best model to unlabeled data
X_unlabeled = df_unlabeled.drop(['class', 'txId'], axis=1)
y_proba = best_model.predict_proba(X_unlabeled)[:, 1]
predictions = (y_proba >= optimal_threshold).astype(int)
df_prediction = pd.Series(predictions, name="prediction")
df_final = pd.concat([df_unlabeled[['txId']], df_prediction], axis=1)
df_final = df_final.applymap(lambda x: 'Illicit' if x == 0 else 'Licit' if x == 1 else x)

# Analyze prediction distribution
class_counts = df_final['prediction'].value_counts()
labeled_only = class_counts[class_counts.index != 'unknown']
imbalance_ratio = labeled_only.max() / labeled_only.min() if len(labeled_only) >= 2 else 1.0

# Plot distribution
print(f"\n📈 Prediction Distribution:")
print(f"Model used:", best_model_name)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
class_counts.plot(kind='bar', ax=ax1, color=['lightblue', 'orange', 'lightcoral'])
ax1.set_title('Class Counts')
ax1.tick_params(axis='x', rotation=45)
class_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=['lightblue', 'orange', 'lightcoral'])
ax2.set_title('Class Distribution')
ax2.set_ylabel('')
plt.tight_layout()
plt.show()

# Display prediction samples and summary statistics
print(f"📊 Prediction Summary:")
print(f"  Total predictions: {len(df_final):,}")
print(f"  Illicit transactions: {sum(df_final['prediction'] == 'Illicit'):,}")
print(f"  Licit transactions: {sum(df_final['prediction'] == 'Licit'):,}")
print(f"  Imbalance ratio: {imbalance_ratio:.1f}:1")

# Get sample transactions for analysis
illicit_selector = df_final['prediction'] == 'Illicit'
X_unlabeled_illicit = df_final[illicit_selector].head(100)
print(f"\n🔮 Sample illicit transactions (showing {len(X_unlabeled_illicit)} of {sum(illicit_selector):,} total)")
display(X_unlabeled_illicit)

licit_selector = df_final['prediction'] == 'Licit'
X_unlabeled_licit = df_final[licit_selector].head(100)
print(f"\n🔮 Sample licit transactions (showing {len(X_unlabeled_licit)} of {sum(licit_selector):,} total)")
display(X_unlabeled_licit)

### Considerations

Some considerations must be made before the conclusion:

- Compressed data can increase training time but reduces dataset sizes without influencing the final result.

- Models like Naive Bayes reached almost 0.6 in contrast to other models and were removed from training;  

- Models like Logistic Regression also had poor performance and were removed from training, but they are used in some ensembles as default estimator models. 

**Future improvements to training:**

- The unsupervised approach could produce good or even better results because it would use much more data to identify patterns by using the complete dataset. Also, the labeled dataset could indicate which clusters could be labeled with the illicit class;

- The training could use models more recommended for graph-type datasets, such as Graph Convolutional Networks (GCN), making use of the edge dataset to learn patterns with deeper transaction chains, not only direct neighbors.

**Production readiness:**

- This training was performed on a dataset curated for research purposes. There is no information about which features were used, so in order to have a production-ready model, a new dataset in the same format would need to be gathered and curated;

- A final performance indicator would need to be established to consider the model ready for a production environment, by classifying real labeled current data.

### Conclusions

#### Model Performance Summary

This supervised learning approach successfully developed a high-performance KYT system achieving **88.49% performance** on cryptocurrency transaction risk classification. The **SVM (Support Vector Machine)** emerged as the champion model, demonstrating superior performance in distinguishing illicit from licit Bitcoin transactions.

#### Key Technical Achievements

- **Dimensionality Reduction**: PCA preprocessing reduced feature space from 166 to 59 dimensions while preserving 95% variance
- **Algorithm Comparison**: Comprehensive evaluation of 10 ML algorithms with hyperparameter optimization via RandomizedSearchCV  
- **Model Ranking**: SVM (88.49%) > KNN (86.87%) > GB (85.02%) demonstrated that ensemble and kernel methods excel in financial pattern recognition
- **Pipeline Standardization**: StandardScaler + PCA + model architecture ensures consistent preprocessing across algorithms
- **Model Persistence**: All trained models saved with compression for deployment scalability
- **Performance Validation**: Stratified cross-validation ensures reliable performance on imbalanced financial data

#### Real-World Impact

The trained model successfully processed **157,205 unlabeled transactions**, identifying **12,675 potentially illicit transactions**, providing risk assessment capabilities for unknown data—essential for AML compliance.